# **Training Deep Neural Networks**

## The Vanishing/Gadients Problem

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# ============================================================================
# 1. HANDLING VANISHING/EXPLODING GRADIENT PROBLEM
# ============================================================================

def create_model_with_proper_initialization(input_shape, num_classes):
    """
    Model with proper weight initialization to handle vanishing/exploding gradients
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=input_shape),

        # BatchNormalization helps with gradient flow
        keras.layers.BatchNormalization(),

        # Dense layer with He initialization for ReLU-like activations
        keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),  # ELU activation helps with gradient flow

        keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),

        # Output layer with appropriate initialization
        keras.layers.Dense(num_classes, activation="softmax",
                          kernel_initializer="glorot_uniform")
    ])
    return model

# Alternative: Using SELU activation with LeCun initialization
def create_selu_model(input_shape, num_classes):
    """
    Model using SELU activation which has self-normalizing properties
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
        keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    return model

# Custom variance scaling initializer
def create_custom_initializer():
    """
    Custom He initialization with average fan mode
    """
    return keras.initializers.VarianceScaling(
        scale=2.0,
        mode='fan_avg',
        distribution='uniform'
    )


## Transfer Learning

In [9]:

# ============================================================================
# 2. REUSING PRETRAINED LAYERS (TRANSFER LEARNING)
# ============================================================================

def create_base_model(input_shape, num_classes):
    """
    Create a base model that can be used for transfer learning
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    return model

def setup_transfer_learning(base_model_path, new_num_classes):
    """
    Setup transfer learning from a pretrained model
    """
    # Load pretrained model
    model_A = keras.models.load_model(base_model_path)

    # Create new model using all layers except the last one
    model_B = keras.models.Sequential(model_A.layers[:-1])

    # Add new output layer for different task
    model_B.add(keras.layers.Dense(new_num_classes, activation="softmax"))

    # Clone the original model for comparison
    model_A_clone = keras.models.clone_model(model_A)
    model_A_clone.set_weights(model_A.get_weights())

    return model_B, model_A_clone

def fine_tune_model(model, X_train, y_train, X_valid, y_valid):
    """
    Two-phase fine-tuning: freeze then unfreeze
    """
    # Phase 1: Freeze all layers except the last one
    for layer in model.layers[:-1]:
        layer.trainable = False

    # Compile and train with frozen layers
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="sgd",
        metrics=["accuracy"]
    )

    print("Phase 1: Training with frozen layers...")
    history_phase1 = model.fit(
        X_train, y_train,
        epochs=4,
        validation_data=(X_valid, y_valid),
        verbose=1
    )

    # Phase 2: Unfreeze all layers and train with lower learning rate
    for layer in model.layers[:-1]:
        layer.trainable = True

    # Use lower learning rate for fine-tuning
    optimizer = keras.optimizers.SGD(learning_rate=1e-4)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    print("Phase 2: Fine-tuning all layers...")
    history_phase2 = model.fit(
        X_train, y_train,
        epochs=16,
        validation_data=(X_valid, y_valid),
        verbose=1
    )

    return history_phase1, history_phase2


## Faster Optimizers

In [10]:

# ============================================================================
# 3. FASTER OPTIMIZERS
# ============================================================================

def get_optimizers():
    """
    Collection of different optimizers with typical hyperparameters
    """
    optimizers = {
        'sgd_momentum': keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
        'rmsprop': keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9),
        'adam': keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
        'sgd_decay': keras.optimizers.SGD(learning_rate=0.01, weight_decay=1e-4)
    }
    return optimizers

# Learning rate scheduling
def exponential_decay_schedule(lr0=0.01, decay_steps=20):
    """
    Exponential decay learning rate schedule
    """
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / decay_steps)
    return exponential_decay_fn

def piecewise_constant_schedule(epoch):
    """
    Piecewise constant learning rate schedule
    """
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

def setup_learning_rate_scheduling():
    """
    Setup different learning rate scheduling strategies
    """
    # Method 1: Using callback with custom function
    exponential_decay_fn = exponential_decay_schedule(lr0=0.01, decay_steps=20)
    lr_scheduler_callback = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

    # Method 2: ReduceLROnPlateau callback
    reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
        factor=0.5,
        patience=5,
        min_lr=1e-6
    )

    # Method 3: Built-in exponential decay schedule
    steps_per_epoch = 1875  # Example: 60000 samples / 32 batch_size
    decay_steps = 20 * steps_per_epoch  # 20 epochs
    exponential_decay_schedule_builtin = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.01,
        decay_steps=decay_steps,
        decay_rate=0.1
    )

    return lr_scheduler_callback, reduce_lr_callback, exponential_decay_schedule_builtin


## Avoiding Overfitting through Regularization

In [11]:

# ============================================================================
# 4. AVOIDING OVERFITTING THROUGH REGULARIZATION
# ============================================================================

# L1 and L2 Regularization
def create_regularized_model(input_shape, num_classes, l2_reg=0.01):
    """
    Model with L2 regularization
    """
    # Create a partial function for regularized dense layer
    RegularizedDense = partial(
        keras.layers.Dense,
        activation="elu",
        kernel_initializer="he_normal",
        kernel_regularizer=keras.regularizers.l2(l2_reg)
    )

    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        RegularizedDense(300),
        RegularizedDense(100),
        RegularizedDense(10, activation="softmax",
                        kernel_initializer="glorot_uniform",
                        kernel_regularizer=None)  # No regularization on output layer
    ])
    return model

# Dropout Regularization
def create_dropout_model(input_shape, num_classes, dropout_rate=0.2):
    """
    Model with dropout regularization
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dropout(rate=dropout_rate),
        keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
        keras.layers.Dropout(rate=dropout_rate),
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
        keras.layers.Dropout(rate=dropout_rate),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    return model

# Monte Carlo Dropout for uncertainty estimation
class MCDropout(keras.layers.Dropout):
    """
    Monte Carlo Dropout layer that applies dropout during inference
    """
    def call(self, inputs, training=None):
        return super().call(inputs, training=True)

def create_mc_dropout_model(input_shape, num_classes, dropout_rate=0.2):
    """
    Model with Monte Carlo Dropout
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        MCDropout(rate=dropout_rate),
        keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
        MCDropout(rate=dropout_rate),
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
        MCDropout(rate=dropout_rate),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    return model

def mc_dropout_prediction(model, X_test, n_samples=100):
    """
    Make predictions using Monte Carlo Dropout
    """
    y_probas = np.stack([
        model(X_test, training=True) for _ in range(n_samples)
    ])
    y_proba = y_probas.mean(axis=0)
    y_std = y_probas.std(axis=0)
    return y_proba, y_std

# Max Norm Constraint
def create_constrained_model(input_shape, num_classes, max_norm=1.0):
    """
    Model with max norm constraint on weights
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dense(300, activation="elu",
                          kernel_initializer="he_normal",
                          kernel_constraint=keras.constraints.max_norm(max_norm)),
        keras.layers.Dense(100, activation="elu",
                          kernel_initializer="he_normal",
                          kernel_constraint=keras.constraints.max_norm(max_norm)),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    return model

# ============================================================================
# 5. COMPLETE TRAINING PIPELINE
# ============================================================================

def create_comprehensive_model(input_shape, num_classes):
    """
    Comprehensive model combining multiple techniques
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=input_shape),

        # Batch normalization for gradient stability
        keras.layers.BatchNormalization(),

        # Regularized dense layer with dropout
        keras.layers.Dense(300,
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01),
                          kernel_constraint=keras.constraints.max_norm(1.0),
                          use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),
        keras.layers.Dropout(0.3),

        keras.layers.Dense(100,
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01),
                          kernel_constraint=keras.constraints.max_norm(1.0),
                          use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),
        keras.layers.Dropout(0.3),

        keras.layers.Dense(num_classes, activation="softmax")
    ])
    return model

def train_comprehensive_model(X_train, y_train, X_valid, y_valid, input_shape, num_classes):
    """
    Complete training pipeline with all techniques
    """
    # Create model
    model = create_comprehensive_model(input_shape, num_classes)

    # Setup optimizer with learning rate scheduling
    initial_learning_rate = 0.001
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=1000,
        decay_rate=0.96,
        staircase=True
    )
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

    # Compile model
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    # Setup callbacks (Remove ReduceLROnPlateau since we're using LearningRateSchedule)
    callbacks = [
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
    ]

    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,  # Reduced epochs for faster training
        batch_size=32,
        validation_data=(X_valid, y_valid),
        callbacks=callbacks,
        verbose=1
    )

    return model, history

def train_comprehensive_model_simple(X_train, y_train, X_valid, y_valid, input_shape, num_classes):
    """
    Simplified training pipeline without learning rate conflicts
    """
    # Create model
    model = create_comprehensive_model(input_shape, num_classes)

    # Use simple Adam optimizer
    optimizer = keras.optimizers.Adam(learning_rate=0.001)

    # Compile model
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    # Setup callbacks with ReduceLROnPlateau
    callbacks = [
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-7),
        keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
    ]

    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_valid, y_valid),
        callbacks=callbacks,
        verbose=1
    )

    return model, history

# ============================================================================
# 6. EXAMPLE USAGE DAN MAIN PROGRAM
# ============================================================================

def example_usage():
    """
    Example of how to use the complete pipeline
    """
    # Load and preprocess data (example with MNIST)
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

    # Normalize data
    X_train = X_train.astype("float32") / 255.0
    X_test = X_test.astype("float32") / 255.0

    # Split validation set
    X_valid, X_train = X_train[:5000], X_train[5000:]
    y_valid, y_train = y_train[:5000], y_train[5000:]

    input_shape = [28, 28]
    num_classes = 10

    print("Training comprehensive model...")
    model, history = train_comprehensive_model(
        X_train, y_train, X_valid, y_valid, input_shape, num_classes
    )

    # Evaluate model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {test_acc:.4f}")

    return model, history

def main():
    """
    Main function - di sini Anda letakkan kode untuk menjalankan model
    """
    print("=== Deep Neural Network Training Pipeline ===\n")

    # 1. Load dan preprocess data
    print("1. Loading and preprocessing data...")
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    X_train = X_train.astype("float32") / 255.0
    X_test = X_test.astype("float32") / 255.0
    X_valid, X_train = X_train[:5000], X_train[5000:]
    y_valid, y_train = y_train[:5000], y_train[5000:]

    print(f"Training data shape: {X_train.shape}")
    print(f"Validation data shape: {X_valid.shape}")
    print(f"Test data shape: {X_test.shape}\n")

    # 2. CONTOH 1: Model dasar dengan teknik anti-overfitting
    print("2. Creating comprehensive model with anti-overfitting techniques...")
    model = create_comprehensive_model([28, 28], 10)
    print("Model architecture:")
    model.summary()
    print()

    # 3. CONTOH 2: Training lengkap dengan semua teknik (versi sederhana)
    print("3. Training model with all advanced techniques...")
    model, history = train_comprehensive_model_simple(
        X_train, y_train, X_valid, y_valid, [28, 28], 10
    )

    # Evaluate final model
    print("4. Evaluating model...")
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Final test accuracy: {test_acc:.4f}\n")

    # 4. CONTOH 3: Transfer Learning (perlu model yang sudah ada)
    print("5. Transfer Learning Example:")
    print("   Note: Untuk transfer learning, Anda perlu model yang sudah dilatih sebelumnya")
    print("   Simpan model saat ini untuk contoh:")

    # Simpan model untuk contoh transfer learning
    model.save("base_model.h5")
    print("   Model saved as 'base_model.h5'")

    # Contoh setup transfer learning
    try:
        print("   Setting up transfer learning...")
        model_B, model_A_clone = setup_transfer_learning("base_model.h5", 2)
        print("   Transfer learning model created successfully!")
        print("   New model architecture for binary classification:")
        model_B.summary()
    except Exception as e:
        print(f"   Transfer learning setup failed: {e}")

    print("\n=== Training Complete ===")

    # 5. CONTOH LAINNYA: Demonstrasi berbagai teknik
    print("\n6. Additional Examples:")

    # Model dengan dropout
    print("   - Dropout model:")
    dropout_model = create_dropout_model([28, 28], 10, dropout_rate=0.3)

    # Model dengan regularisasi L2
    print("   - L2 Regularized model:")
    l2_model = create_regularized_model([28, 28], 10, l2_reg=0.01)

    # Model dengan SELU activation
    print("   - SELU model:")
    selu_model = create_selu_model([28, 28], 10)

    # Monte Carlo Dropout
    print("   - Monte Carlo Dropout model:")
    mc_model = create_mc_dropout_model([28, 28], 10)

    print("   All model variants created successfully!")

    return model, history

# ============================================================================
# JALANKAN PROGRAM UTAMA
# ============================================================================

if __name__ == "__main__":
    # CARA 2: Atau jalankan contoh-contoh spesifik di bawah ini:

    # Load data terlebih dahulu
     (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
     X_train = X_train.astype("float32") / 255.0
     X_test = X_test.astype("float32") / 255.0
     X_valid, X_train = X_train[:5000], X_train[5000:]
     y_valid, y_train = y_train[:5000], y_train[5000:]

    # Untuk model dasar dengan teknik anti-overfitting
     model = create_comprehensive_model([28, 28], 10)
     print("Comprehensive model created!")

    # Untuk training lengkap dengan semua teknik
     model, history = train_comprehensive_model(X_train, y_train, X_valid, y_valid, [28, 28], 10)
     print("Model training completed!")

     # Untuk transfer learning (pastikan ada base_model.h5)
     # model_B, model_A_clone = setup_transfer_learning("base_model.h5", 2)
     # print("Transfer learning setup completed!")

Comprehensive model created!
Epoch 1/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8363 - loss: 1.7799

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.8363 - loss: 1.7795 - val_accuracy: 0.9080 - val_loss: 0.5857
Epoch 2/50
1704/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8857 - loss: 0.6392

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.8857 - loss: 0.6391 - val_accuracy: 0.9210 - val_loss: 0.5372
Epoch 3/50
1706/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8922 - loss: 0.6061

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8922 - loss: 0.6060 - val_accuracy: 0.9258 - val_loss: 0.4922
Epoch 4/50
1705/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9003 - loss: 0.5711

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9003 - loss: 0.5710 - val_accuracy: 0.9290 - val_loss: 0.4831
Epoch 5/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9048 - loss: 0.5411

...

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9714 - loss: 0.1371 - val_accuracy: 0.9740 - val_loss: 0.1340
Epoch 43/50
1700/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9709 - loss: 0.1368

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9709 - loss: 0.1368 - val_accuracy: 0.9764 - val_loss: 0.1310
Epoch 44/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9698 - loss: 0.1331 - val_accuracy: 0.9764 - val_loss: 0.1324
Epoch 45/50
1716/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9737 - loss: 0.1289

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9737 - loss: 0.1289 - val_accuracy: 0.9756 - val_loss: 0.1264
Epoch 46/50
1712/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9721 - loss: 0.1260

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9721 - loss: 0.1260 - val_accuracy: 0.9776 - val_loss: 0.1232
Epoch 47/50
1713/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9747 - loss: 0.1205

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9747 - loss: 0.1205 - val_accuracy: 0.9760 - val_loss: 0.1166
Epoch 48/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9761 - loss: 0.1151 - val_accuracy: 0.9752 - val_loss: 0.1200
Epoch 49/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9775 - loss: 0.1121 - val_accuracy: 0.9764 - val_loss: 0.1174
Epoch 50/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9766 - loss: 0.1097 - val_accuracy: 0.9760 - val_loss: 0.1172
Model training completed!
